<a href="https://colab.research.google.com/github/Anyaoma/modernpython/blob/master/Options_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EUROPEAN AND AMERICAN OPTION PRICING UNDER A TRINOMIAL TREE

In [1]:
import numpy as np

# European Options Pricing Using The Trinomial Tree

In [40]:
class TrinomialModel():
  def __init__(self,s0, rf, sigma,maturity):
    self.__rf = rf
    self.__s0 = s0
    self.__sigma = sigma
    self.__T = maturity


  def __up_and_down_factor(self,up =None):
    if up == None:
      up = np.exp(self.__sigma * np.sqrt(2*self.__h))

    assert up > 0.0 , f'This number {up} should be non negative'

    down = 1/up

    assert down < up

    self.__up = up
    self.__down = down

  def __risk_n_probs(self):
    self.__pu = ((np.exp(self.__rf * self.__h / 2)
    - np.exp(- self.__sigma * np.sqrt(self.__h / 2)))

                 /

    (np.exp(self.__sigma * np.sqrt(self.__h / 2)) -
    np.exp(-self.__sigma * np.sqrt(self.__h/2))))**2

    self.__pd = (
            (
                -np.exp(self.__rf * self.__h / 2)
                + np.exp(self.__sigma * np.sqrt(self.__h / 2))
            )
            / (
                np.exp(self.__sigma * np.sqrt(self.__h / 2))
                - np.exp(-self.__sigma * np.sqrt(self.__h / 2))
            )
        ) ** 2

    self.__pm = 1- self.__pu - self.__pd

    assert 0 <= self.__pu <= 1.0
    assert 0 <= self.__pd <= 1.0
    assert 0 <= self.__pm <= 1.0


  def __terminal_stock(self, nperiod):
    stock_up = self.__up * np.ones(nperiod)
    np.cumproduct(stock_up, out=stock_up)

    stock_down = self.__down * np.ones(nperiod)
    np.cumproduct(stock_down, out=stock_down)

    comb_data = np.concatenate((stock_down[::-1],[1.0],stock_up))
    comb_data *= self.__s0

    return comb_data


  def payoff(self, stock_terminal): #it will be used by other methods in this class
    raise NotImplementedError

  def current_price(self, current_stock, next_opt_payoff):
    expected_opt = np.zeros(current_stock.size)
    for i in range(expected_opt.size):
      p = next_opt_payoff[i] * self.__pd
      p += next_opt_payoff[i+1] * self.__pm
      p += next_opt_payoff[i+2] * self.__pu

      expected_opt[i] = p

    return self.__discount * expected_opt

  def option_price(self, nperiods, up=None):
    assert nperiods > 0
    nperiods = int(nperiods)

    self.__h = self.__T / nperiods
    self.__up_and_down_factor(up)
    self.__risk_n_probs()

    self.__discount = np.exp(-self.__rf * self.__h)

    terminal_s_price = self.__terminal_stock(nperiods)
    final_payoff = self.payoff(terminal_s_price)
    next_option_payoff = final_payoff

    for i in range(1, nperiods + 1):
      vec_stock = self.__terminal_stock(nperiods - i)
      next_option_payoff = self.current_price(vec_stock, next_option_payoff)

    return next_option_payoff[0]



In [41]:
class EuropianTrinomialCall(TrinomialModel):
  def __init__(self,s0, rf, sigma,maturity,K):
    super(EuropianTrinomialCall, self).__init__(s0, rf, sigma,maturity)
    self.__K = K

  def payoff(self,s):
    return np.maximum(s-self.__K, 0.0)



In [78]:
class EuropianTrinomialPut(TrinomialModel):
  def __init__(self,s0, rf, sigma,maturity,K):
    super(EuropianTrinomialPut, self).__init__(s0, rf, sigma, maturity)
    self.__K = K


  def payoff(self,s):
    return np.maximum(self.__K  - s, 0.0)

In [79]:
Eur_call = EuropianTrinomialCall(100.0, 0.0, 0.3,1.0,90.0)

In [80]:
print(Eur_call.option_price(2))

17.50162310051333


<ipython-input-40-e1234b698604>:86: DeprecationWarning: `cumproduct` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `cumprod` instead.
  terminal_s_price = self.__terminal_stock(nperiods)
<ipython-input-40-e1234b698604>:91: DeprecationWarning: `cumproduct` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `cumprod` instead.
  vec_stock = self.__terminal_stock(nperiods - i)


In [81]:
Eur_put = EuropianTrinomialPut(100.0, 0.0, 0.3,1.0,90.0)

In [82]:
print(Eur_put.option_price(2))

7.501623100513319


<ipython-input-40-e1234b698604>:86: DeprecationWarning: `cumproduct` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `cumprod` instead.
  terminal_s_price = self.__terminal_stock(nperiods)
<ipython-input-40-e1234b698604>:91: DeprecationWarning: `cumproduct` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `cumprod` instead.
  vec_stock = self.__terminal_stock(nperiods - i)


## put-call parity test

In [88]:
call = Eur_call.option_price(2)
put = Eur_put.option_price(2)

<ipython-input-40-e1234b698604>:86: DeprecationWarning: `cumproduct` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `cumprod` instead.
  terminal_s_price = self.__terminal_stock(nperiods)
<ipython-input-40-e1234b698604>:91: DeprecationWarning: `cumproduct` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `cumprod` instead.
  vec_stock = self.__terminal_stock(nperiods - i)


In [90]:
#put-call parity
round(call,2) + 90*np.exp(-0.0*1) == 100 + round(put,2)

True

# American Options pricing using the Trinomial Tree

In [56]:
class AmericanTrinomialModel():
  def __init__(self,s0, rf, sigma,maturity,K):
    self.__K = K
    self.__rf = rf
    self.__s0 = s0
    self.__sigma = sigma
    self.__T = maturity


  def __up_and_down_factor(self,up =None):
    if up == None:
      up = np.exp(self.__sigma * np.sqrt(2*self.__h))

    assert up > 0.0 , f'This number {up} should be non negative'

    down = 1/up

    assert down < up

    self.__up = up
    self.__down = down

  def __risk_n_probs(self):
    self.__pu = ((np.exp(self.__rf * self.__h / 2)
    - np.exp(- self.__sigma * np.sqrt(self.__h / 2)))

                 /

    (np.exp(self.__sigma * np.sqrt(self.__h / 2)) -
    np.exp(-self.__sigma * np.sqrt(self.__h/2))))**2

    self.__pd = (
            (
                -np.exp(self.__rf * self.__h / 2)
                + np.exp(self.__sigma * np.sqrt(self.__h / 2))
            )
            / (
                np.exp(self.__sigma * np.sqrt(self.__h / 2))
                - np.exp(-self.__sigma * np.sqrt(self.__h / 2))
            )
        ) ** 2

    self.__pm = 1- self.__pu - self.__pd

    assert 0 <= self.__pu <= 1.0
    assert 0 <= self.__pd <= 1.0
    assert 0 <= self.__pm <= 1.0


  def __terminal_stock(self, nperiod):
    stock_up = self.__up * np.ones(nperiod)
    np.cumproduct(stock_up, out=stock_up)

    stock_down = self.__down * np.ones(nperiod)
    np.cumproduct(stock_down, out=stock_down)

    comb_data = np.concatenate((stock_down[::-1],[1.0],stock_up))
    comb_data *= self.__s0

    return comb_data


  def payoff(self, stock_terminal): #it will be used by other methods in this class
    raise NotImplementedError

  def current_price(self, current_stock, next_opt_payoff):
    expected_opt = np.zeros(current_stock.size)
    for i in range(expected_opt.size):
      p = next_opt_payoff[i] * self.__pd
      p += next_opt_payoff[i+1] * self.__pm
      p += next_opt_payoff[i+2] * self.__pu

      expected_opt[i] = np.maximum(p,np.maximum((current_stock[i] - self.__K),0.0) )

    return self.__discount * expected_opt

  def option_price(self, nperiods, up=None):
    assert nperiods > 0
    nperiods = int(nperiods)

    self.__h = self.__T / nperiods
    self.__up_and_down_factor(up)
    self.__risk_n_probs()

    self.__discount = np.exp(-self.__rf * self.__h)

    terminal_s_price = self.__terminal_stock(nperiods)
    final_payoff = self.payoff(terminal_s_price)
    next_option_payoff = final_payoff

    for i in range(1, nperiods + 1):
      vec_stock = self.__terminal_stock(nperiods - i)
      next_option_payoff = self.current_price(vec_stock, next_option_payoff)

    return next_option_payoff[0]


In [62]:
class AmericanTrinomialCall(AmericanTrinomialModel):
  def __init__(self,s0, rf, sigma,maturity,K):
    super(AmericanTrinomialCall, self).__init__(s0, rf, sigma,maturity,K)
    self.__K = K

  def payoff(self,s):
    return np.maximum(s-self.__K, 0.0)


In [83]:
Amer_call = AmericanTrinomialCall(100.0, 0.0, 0.3,1.0,90.0)

In [84]:
print(Amer_call.option_price(2))

17.50162310051333


<ipython-input-56-b2b0c7a4abca>:87: DeprecationWarning: `cumproduct` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `cumprod` instead.
  terminal_s_price = self.__terminal_stock(nperiods)
<ipython-input-56-b2b0c7a4abca>:92: DeprecationWarning: `cumproduct` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `cumprod` instead.
  vec_stock = self.__terminal_stock(nperiods - i)


In [85]:
class AmericanTrinomialPut(AmericanTrinomialModel):
  def __init__(self,s0, rf, sigma,maturity,K):
    super(AmericanTrinomialPut, self).__init__(s0, rf, sigma, maturity,K)
    self.__K = K

  def payoff(self,s):
    return np.maximum(self.__K  - s, 0.0)

In [86]:
Amer_put = AmericanTrinomialPut(100.0, 0.0, 0.3,1.0,90.0)

In [87]:
print(Amer_put.option_price(2))

19.813336025386302


<ipython-input-56-b2b0c7a4abca>:87: DeprecationWarning: `cumproduct` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `cumprod` instead.
  terminal_s_price = self.__terminal_stock(nperiods)
<ipython-input-56-b2b0c7a4abca>:92: DeprecationWarning: `cumproduct` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `cumprod` instead.
  vec_stock = self.__terminal_stock(nperiods - i)
